In [1]:
import os
from nbed.driver import NbedDriver
import numpy as np

from nbed.ham_builder import HamiltonianBuilder
from openfermion import get_sparse_operator
import scipy as sp
    
from pyscf import gto, scf, fci, cc

In [2]:
# get xyz file for water

notebook_dir = os.getcwd()
source_dir = os.path.dirname(notebook_dir)
docs_dir = os.path.dirname(source_dir)
NBed_dir = os.path.dirname(docs_dir)
Test_dir = os.path.join(NBed_dir, 'tests')
mol_dir = os.path.join(Test_dir, 'molecules')

water_xyz_path = os.path.join(mol_dir, 'water.xyz')

In [3]:

with open(water_xyz_path, 'r') as infile:
    xyz_string = infile.read()
    
print(xyz_string)

3

O   0.00000  0.0000000  0.1653507
H   0.00000  0.7493682 -0.4424329
H   0.00000 -0.7493682 -0.4424329 


In [4]:
# options
geometry = xyz_string
n_active_atoms=1
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
run_virtual_localization = False

run_fci_emb = False
run_ccsd_emb = False
max_ram_memory = 8_000
_init_huzinaga_rhf_with_mu = False

max_hf_cycles=5000


projector = 'both' # developer flag that runs both huzinaga and mu shift
localization = 'spade' # spade, ibo
occupied_threshold = 0.99
virtual_threshold = 0.95

In [5]:
driver = NbedDriver(geometry = geometry,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    max_ram_memory = max_ram_memory,
                    init_huzinaga_rhf_with_mu = _init_huzinaga_rhf_with_mu,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization)

1970-01-03 17:06:58,137: nbed.driver: WARNING: Outputting both mu and huzinaga embedding results as tuple.


# FCI calc

In [6]:
charge=0
spin=0

full_system_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.build()

global_rhf = scf.RHF(full_system_mol)
global_rhf.verbose=1
global_rhf.conv_tol = 1e-6
global_rhf.kernel()


## run FCI embedded
glob_FCI = fci.FCI(global_rhf)
glob_FCI.run()

print(f'full FCI energy: {glob_FCI.e_tot}')

full FCI energy: -75.01535247852486


# 1. Low-level whole system calculation

The first step is to run a DFT caluclation of the whole system.

In [7]:
### inputs

charge=0
spin=0

full_system_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.build()

global_rks = scf.RKS(full_system_mol)
global_rks.verbose=1
global_rks.xc = xc_functional
global_rks.conv_tol = 1e-6
global_rks.kernel()

global_rks_Fock = global_rks.get_fock()
c_global_rks_full = global_rks.mo_coeff
global_rks_etot = global_rks.energy_tot()
global_rks.mo_energy = np.diag(c_global_rks_full.conj().T @ global_rks_Fock@ c_global_rks_full)
global_rks_mo_energies = global_rks.mo_energy 
global_rks_mo_energies

array([-18.83525318,  -0.93132671,  -0.43263415,  -0.23435814,
        -0.14179089,   0.35582625,   0.46190031])

In [8]:
# np.around(c_global_rks_full.conj().T @ global_rks_Fock@ c_global_rks_full, 5)

In [9]:
# checking global DFT calc
c_occ = global_rks.mo_coeff[:, global_rks.mo_occ>0]
dmat = 2* c_occ @ c_occ.T
e_tot = global_rks.energy_nuc() + global_rks.energy_elec(dm =dmat)[0]

np.isclose(global_rks.e_tot, e_tot)

True

In [10]:
# getting matrices manual
two_veff = global_rks.get_veff(dm=dmat)
vj = two_veff.vj
vk = two_veff.vk 

In [11]:
# calc xc contribution (DFT) via manual integral
ni = global_rks._numint
nelec, exc, vxc = ni.nr_rks(global_rks.mol, global_rks.grids, global_rks.xc, global_rks.make_rdm1())
exc - 0.25 * np.einsum('ij, ji ->', vk, dmat) 

-9.408825304427083

In [12]:
# check if this gives the same results
np.einsum('ij, ji ->',-0.25*vk, dmat) + exc

-9.408825304427083

In [13]:
two_veff.exc

-9.408825304427083

In [14]:
# two_veff construction
np.allclose(two_veff, vj - 0.5*vk + vxc)

True

In [15]:
print(two_veff.ecoul)
print(0.5 * np.einsum('ij, ji ->', vj, dmat) )

47.23189642113185
47.23189642113185


In [16]:
print(two_veff.exc + two_veff.ecoul)
print(0.5 * np.einsum('ij, ji ->', vj - 0.5*vk, dmat)+ exc)

37.82307111670477
37.82307111670477


In [17]:
## DFT fock matrix
F_obj = global_rks.get_fock()
F_manual = two_veff + global_rks.get_hcore()

np.allclose(F_obj, F_manual)

True

In [18]:
FF =  0.5*(vj - 0.5*vk) + global_rks.get_hcore()
np.einsum('ij, ji ->',FF, dmat) + exc

-84.40616856232964

In [19]:
np.allclose(two_veff, vj - 0.5*vk + vxc)

True

In [20]:
# subtract out the vxc part!!! (aka only leaving: 0.5*(vj - 0.5*vk) part!)

FF2 =  0.5*(two_veff - vxc)  + global_rks.get_hcore()
np.einsum('ij, ji ->',FF2, dmat) + exc

-84.40616856232964

In [21]:
global_rks.energy_elec()[0]

-84.4061685623296

# 2. Localize

We then use this object to find LOCALIZED systems (active and enviroment)

(more details given in notebook 2)

In [22]:
from nbed.localizers import (
    BOYSLocalizer,
    IBOLocalizer,
    Localizer,
    PMLocalizer,
    SPADELocalizer,
)

run_virtual_localization = False


localizers = {
    'boys':BOYSLocalizer,
    'ibo':IBOLocalizer,
    'pipek-mezey': PMLocalizer,
    'spade': SPADELocalizer
}

In [23]:
### inputs

charge=0
spin=0

full_system_mol2 = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol2.build()

global_rks2 = scf.RKS(full_system_mol2)
global_rks2.verbose=1
global_rks2.xc = xc_functional
global_rks2.conv_tol = 1e-6
global_rks2.kernel()

global_rks_Fock = global_rks2.get_fock()
c_global_rks_full = global_rks2.mo_coeff
global_rks_etot = global_rks2.energy_tot()
global_rks2.mo_energy = np.diag(c_global_rks_full.conj().T @ global_rks_Fock@ c_global_rks_full)

In [24]:
localized_system = localizers[localization](global_rks2,
                                        n_active_atoms,
                                        occ_cutoff=occupied_threshold,
                                        virt_cutoff=virtual_threshold,
                                        run_virtual_localization=run_virtual_localization,
)

In [25]:
print(f'active inds: {localized_system.active_MO_inds}')
print(f'enviro inds: {localized_system.enviro_MO_inds}')

active inds: [0 1 2]
enviro inds: [3 4]


# 3. Run subsystem DFT (uses localized rks)

In [26]:
def rks_components(
    RKS_obj, subsystem_dm: np.ndarray):
    """Calculate the components of subsystem energy from a RKS DFT calculation.

    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.

    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)

    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    dm_matrix = subsystem_dm
    # It seems that PySCF lumps J and K in the J array
    two_e_term = RKS_obj.get_veff(dm=dm_matrix)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", RKS_obj.get_hcore(), dm_matrix)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    return energy_elec, e_xc, j_mat

In [27]:
(e_act, e_xc_act, j_act) = rks_components(global_rks, localized_system.dm_active)

(e_env, e_xc_env, j_env) = rks_components(global_rks, localized_system.dm_enviro)

# Computing cross subsystem terms
two_e_term_total = global_rks.get_veff(
    dm=localized_system.dm_active + localized_system.dm_enviro
)
e_xc_total = two_e_term_total.exc

j_cross = 0.5 * (
    np.einsum("ij,ij", localized_system.dm_active, j_env)
    + np.einsum("ij,ij", localized_system.dm_enviro, j_act)
)
# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env

# overall two_electron cross energy
two_e_cross = j_cross + k_cross + xc_cross

energy_DFT_components = (e_act + e_env + two_e_cross + global_rks.energy_nuc())

if not np.isclose(energy_DFT_components, global_rks.e_tot):
    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT")


In [28]:
# e_act, e_env, two_e_cross, localized_system, j_env = subsystem_dft(localized_system, localized_system.rks)

# DFT energy made from component parts
E_dft_from_components = e_act + e_env + two_e_cross + global_rks.energy_nuc()
np.isclose(global_rks.e_tot, E_dft_from_components)

True

In [29]:
dm_full = global_rks.make_rdm1()

full = global_rks.get_veff(dm=dm_full)
# print(full.exc + full.ecoul)
A= global_rks.get_veff(dm=localized_system.dm_active)
B= global_rks.get_veff(dm=localized_system.dm_enviro)
non_add = (full.exc + full.ecoul) - (A.exc + A.ecoul + B.exc + B.ecoul)

np.isclose(two_e_cross, non_add)

True

In [30]:
A_energy = np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_active) + (A.exc + A.ecoul)
B_energy = np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_enviro) + (B.exc + B.ecoul)
components = A_energy + B_energy + non_add + global_rks.energy_nuc()
np.isclose(global_rks.e_tot, components)

True

In [31]:
j_active = A.vj
E_manual = np.einsum("ij,ji->", global_rks.get_hcore() + j_active/2, localized_system.dm_active) + A.exc
np.isclose(A_energy, E_manual)

True

In [32]:
EE = (np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_active) +
      np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_enviro) +
      full.exc + full.ecoul
      + global_rks.energy_nuc()
    )
np.isclose(global_rks.e_tot, EE)

True

# 4. Get DFT potential

In [33]:
g_act_and_env = global_rks.get_veff(
    dm=(localized_system.dm_active + localized_system.dm_enviro)
)
g_act = global_rks.get_veff(dm=localized_system.dm_active)
dft_potential = g_act_and_env - g_act # + j_env

In [34]:
# localized_system.rks
dmat = 2* c_occ @ c_occ.T
print(np.allclose(dmat, localized_system.dm_active + localized_system.dm_enviro))

print(np.allclose(g_act_and_env, two_veff))

True
True


In [35]:
# vj_act = g_act.vj
# vk_act= g_act.vk

# ni = global_rks._numint
# nelec_act, exc_act, vxc_act = ni.nr_rks(global_rks.mol,
#                                         global_rks.grids, 
#                                         global_rks.xc,
#                                         localized_system.dm_active)


# np.allclose(g_act, (vj_act - 0.5*vk_act) + vxc_act)

dft_potential is

$$DFT\: potential = g[\gamma_{act} + \gamma_{env}] - g[\gamma_{act}]$$

In [36]:
rks_emb_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
rks_emb_mol.nelectron = 2 * len(localized_system.active_MO_inds) # <--change no e-
rks_emb_mol.build()

rks_emb = scf.RKS(rks_emb_mol)
rks_emb.verbose=1
rks_emb.xc = xc_functional
rks_emb.conv_tol = 1e-6

s_mat = global_rks.get_ovlp()
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat

h_core_Std = rks_emb.get_hcore()
rks_emb.get_hcore = lambda *args: h_core_Std + 1e6*enviro_projector + dft_potential

rks_emb.kernel()

-51.99066545718777

In [37]:
y_emb =  rks_emb.make_rdm1()
diff_gamma = y_emb - localized_system.dm_active

In [38]:
e_act

-77.37494139936763

In [39]:
(global_rks.energy_elec(dm=y_emb)[0]  
 + np.einsum("ij,ij->",  dft_potential+1e6*enviro_projector, diff_gamma) 
 )

-77.37494171952797

# WF approach (mu shift projector)

In [40]:
charge=0
spin=0

full_system_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.nelectron = 2 * len(localized_system.active_MO_inds) # <- change number of e-
full_system_mol.build()

# HARTREE FOCK!
scf_method = scf.RHF(full_system_mol)
scf_method.verbose=1
scf_method.conv_tol = 1e-6

In [41]:
# scf_method = init_localised_RHF_obj(geometry, basis, charge, localized_system)
mu_level_shift = 1e6

# redefine core H
std_hcore = scf_method.get_hcore()
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat
scf_method.get_hcore = lambda *args: std_hcore + mu_level_shift*enviro_projector + dft_potential

scf_method.kernel()

-51.831019568708186

In [42]:
correction = np.einsum("ij,ji", mu_level_shift*enviro_projector + dft_potential,
                       localized_system.dm_active) # <--- different 
correction

16.25599253414445

In [43]:
scf_method.energy_elec()[0] - correction

-77.21529583113558

In [44]:
e_act

-77.37494139936763

In [45]:
## run FCI embedded
fci_scf_MU_embedded = fci.FCI(scf_method)

fci_scf_MU_embedded.run()

fci_scf_MU_embedded.e_tot

-51.835896277637445

In [46]:
e_corr = fci_scf_MU_embedded.e_tot - scf_method.e_tot
e_corr

-0.004876708929259621

In [47]:
E_emb_act = scf_method.energy_elec()[0] - correction + e_corr 
E_emb_full = E_emb_act + e_env + two_e_cross + global_rks.energy_nuc()
E_emb_full

-75.12311597474392

In [48]:
print(f'global DFT energy: {global_rks.e_tot}')
print(f'global FCI energy: {glob_FCI.e_tot}')
print()

print(f'emb error : {abs(E_emb_full - glob_FCI.e_tot)}')
print(f'DFT error : {abs(global_rks.e_tot - glob_FCI.e_tot)}')

global DFT energy: -75.27788483404667
global FCI energy: -75.01535247852486

emb error : 0.10776349621906434
DFT error : 0.2625323555218131


# WF approach (huzinaga)

In [49]:
charge=0
spin=0

full_system_mol_HUZ = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_HUZ.nelectron = 2 * len(localized_system.active_MO_inds) # <- change number of e-
full_system_mol_HUZ.build()

# HARTREE FOCK!
scf_method_HUZ = scf.RHF(full_system_mol_HUZ)
scf_method_HUZ.verbose=1
scf_method_HUZ.conv_tol = 1e-6

In [50]:
from nbed.scf import huzinaga_RHF, huzinaga_RKS

In [51]:
# see source code 
# implements manual RHF calc with Huzinaga projector included

(c_active_embedded, 
 mo_energy, 
 dm_active_embedded,
 huzinaga_op_std,
 huz_rhf_conv_flag) =huzinaga_RHF(
    scf_method_HUZ,
    dft_potential,
    localized_system.dm_enviro,
    dm_conv_tol=1e-6,
    dm_initial_guess=None,
    use_DIIS= False,
)

hcore_std = scf_method_HUZ.get_hcore()
v_emb = huzinaga_op_std + dft_potential
scf_method_HUZ.get_hcore = lambda *args: hcore_std + v_emb
scf_method_HUZ.mo_coeff = c_active_embedded
scf_method_HUZ.mo_occ = scf_method_HUZ.get_occ(
    mo_energy, c_active_embedded
)
scf_method_HUZ.mo_energy = mo_energy
scf_method_HUZ.conv_check = huz_rhf_conv_flag

scf_method_HUZ.e_tot = scf_method_HUZ.energy_tot(dm=dm_active_embedded)


In [52]:
mo_energy

array([-20.22912626,  -0.71962271,  -0.36944532,   0.15734971,
         0.4309262 ,   0.6325906 ,   0.72378528])

In [53]:
overlap = np.einsum('ij, ki -> i', c_active_embedded.T,  huzinaga_op_std @ c_active_embedded)
overlap

array([ 5.91360847e-01,  4.11204883e-01, -1.17427060e-16,  1.44475353e-01,
        6.84298537e-01,  2.54867669e-01, -7.54246096e-02])

In [54]:
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat
overlap = np.einsum('ij, ki -> i', c_active_embedded.T,  enviro_projector @ c_active_embedded)

overlap_by_size = overlap.argsort()[::-1]

n_env_mo = len(localized_system.enviro_MO_inds)
frozen_enviro_orb_inds = overlap_by_size[:n_env_mo]
active_MOs_occ_and_virt_embedded = [
                mo_i
                for mo_i in range(c_active_embedded.shape[1])
                if mo_i not in frozen_enviro_orb_inds
            ]

c_selected = c_active_embedded[:, active_MOs_occ_and_virt_embedded]
mo_energy_act = np.diag(c_selected.conj().T @ scf_method_HUZ.get_fock()@ c_selected)
mo_energy_act

array([-20.22912626,  -0.71962272,  -0.36944533,   0.6325906 ,
         0.72378527])

In [55]:
np.around(c_selected.conj().T @ scf_method_HUZ.get_fock()@ c_selected, 3)

array([[-20.229,  -0.   ,   0.   ,   0.   ,  -0.   ],
       [ -0.   ,  -0.72 ,  -0.   ,   0.   ,   0.   ],
       [  0.   ,  -0.   ,  -0.369,  -0.   ,   0.   ],
       [  0.   ,   0.   ,  -0.   ,   0.633,  -0.   ],
       [ -0.   ,   0.   ,   0.   ,  -0.   ,   0.724]])

In [56]:
print(scf_method.mo_energy)

[-2.02291263e+01 -7.19622452e-01 -3.69445118e-01  6.32590516e-01
  7.23785410e-01  1.99999957e+06  1.99999984e+06]


In [57]:
## run FCI embedded
scf_method_HUZ.mo_coeff = c_active_embedded[:, active_MOs_occ_and_virt_embedded]

fci_scf_HUZ_embedded = fci.FCI(scf_method_HUZ)

fci_scf_HUZ_embedded.run()
fci_scf_HUZ_embedded.e_tot

-51.835895926036876

In [58]:
e_corr_HUZ = fci_scf_HUZ_embedded.e_tot - scf_method_HUZ.e_tot
e_corr_HUZ

-0.004876696459945151

In [59]:
correction_HUZ = np.einsum("ij,ji", v_emb,
                       localized_system.dm_active) # <--- different 
correction_HUZ

16.255992534510348

In [60]:
E_emb_act_HUZ = scf_method_HUZ.energy_elec(dm=dm_active_embedded)[0] - correction + e_corr_HUZ 
E_emb_full_HUZ = E_emb_act_HUZ + e_env + two_e_cross + global_rks.energy_nuc()


print(f'emb error : {abs(E_emb_full_HUZ - glob_FCI.e_tot)}')
print(f'DFT error : {abs(global_rks.e_tot - glob_FCI.e_tot)}')

emb error : 0.10776314461850234
DFT error : 0.2625323555218131


In [61]:
print(f'HUZ vs MU : {abs(E_emb_full - E_emb_full_HUZ)}')

HUZ vs MU : 3.5160056199856626e-07


# Looking at localized system

In [62]:
print(np.around(localized_system.c_enviro, 3))

[[ 0.101  0.   ]
 [-0.353 -0.   ]
 [ 0.    -0.   ]
 [ 0.    -0.611]
 [ 0.557  0.   ]
 [-0.296 -0.446]
 [-0.296  0.446]]


In [63]:
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat

print(np.around(enviro_projector@ localized_system.c_loc_occ_and_virt, 3))

[[-0.     0.    -0.    -0.028  0.     0.     0.   ]
 [-0.     0.     0.    -1.213 -0.     0.    -0.   ]
 [-0.     0.    -0.     0.    -0.    -0.     0.   ]
 [-0.     0.     0.     0.    -1.764 -0.     0.   ]
 [ 0.    -0.    -0.     1.406  0.    -0.     0.   ]
 [-0.     0.     0.    -1.339 -1.033  0.     0.   ]
 [ 0.     0.     0.    -1.339  1.033  0.    -0.   ]]


In [64]:
proj_env = localized_system.dm_enviro/2 @ s_mat

print(np.around(localized_system.c_enviro, 3))
print()
print(np.around(proj_env @ localized_system.c_loc_occ_and_virt, 3))


[[ 0.101  0.   ]
 [-0.353 -0.   ]
 [ 0.    -0.   ]
 [ 0.    -0.611]
 [ 0.557  0.   ]
 [-0.296 -0.446]
 [-0.296  0.446]]

[[ 0.    -0.    -0.     0.101  0.    -0.     0.   ]
 [-0.     0.     0.    -0.353 -0.     0.    -0.   ]
 [-0.     0.    -0.     0.    -0.    -0.     0.   ]
 [-0.     0.    -0.     0.    -0.611 -0.     0.   ]
 [ 0.    -0.    -0.     0.557  0.    -0.     0.   ]
 [-0.     0.     0.    -0.296 -0.446  0.     0.   ]
 [ 0.     0.    -0.    -0.296  0.446  0.    -0.   ]]


In [65]:
proj_env2 = s_mat @ localized_system.dm_enviro/2 # different order!
np.allclose(proj_env.conj().T, proj_env2)

True

# Get projector onto enviroment

In [66]:
s_mat = global_rks.get_ovlp()
s_half = sp.linalg.fractional_matrix_power(s_mat, 0.5)
s_neg_half = sp.linalg.fractional_matrix_power(s_mat, -0.5)

# 1. Get orthogonal C matrix (localized)
c_loc_ortho = s_half @ localized_system.c_loc_occ_and_virt

# 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
#    (do this in orthongoal basis!)
#    note we only take MO environment indices!
ortho_proj = np.einsum(
    "ik,jk->ij",
    c_loc_ortho[:, localized_system.enviro_MO_inds],
    c_loc_ortho[:, localized_system.enviro_MO_inds],
)

In [67]:
# note no two times for this dm!!!
dm = localized_system.c_enviro @ localized_system.c_enviro.T
ortho_proj2 =  s_half @ dm @ s_half
np.allclose(ortho_proj, ortho_proj2)

True

In [68]:
enviro_projector = s_neg_half @ ortho_proj @ s_half
print(np.around(enviro_projector@localized_system.c_enviro, 5))
print()

enviro_projector = dm @ s_mat
print(np.around(enviro_projector@localized_system.c_enviro, 5))
print()

print(np.around(localized_system.c_enviro, 5))

[[ 0.10066  0.     ]
 [-0.35251 -0.     ]
 [ 0.      -0.     ]
 [ 0.      -0.6113 ]
 [ 0.55729  0.     ]
 [-0.29575 -0.44592]
 [-0.29575  0.44592]]

[[ 0.10066  0.     ]
 [-0.35251 -0.     ]
 [ 0.      -0.     ]
 [ 0.      -0.6113 ]
 [ 0.55729  0.     ]
 [-0.29575 -0.44592]
 [-0.29575  0.44592]]

[[ 0.10066  0.     ]
 [-0.35251 -0.     ]
 [ 0.      -0.     ]
 [ 0.      -0.6113 ]
 [ 0.55729  0.     ]
 [-0.29575 -0.44592]
 [-0.29575  0.44592]]


In [69]:
enviro_projector = s_mat@ dm @ s_mat # paper definition!
print(np.around(enviro_projector@localized_system.c_active, 10))

[[-0.  0. -0.]
 [-0.  0.  0.]
 [-0.  0. -0.]
 [-0.  0.  0.]
 [ 0. -0. -0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]


In [70]:
np.trace(s_mat@ localized_system.dm_enviro) # @ s_mat)

3.999999999999999

In [71]:
proj_act = localized_system.dm_active/2 @ s_mat # uses active dmat!

print(np.around(localized_system.c_active, 3))
print()
print(np.around(proj_act @ localized_system.c_loc_occ_and_virt, 3))

[[-0.226 -0.002 -0.996]
 [ 0.945  0.008 -0.021]
 [-0.009  1.    -0.   ]
 [ 0.    -0.    -0.   ]
 [ 0.53   0.005 -0.01 ]
 [-0.148 -0.001  0.015]
 [-0.148 -0.001  0.015]]

[[-0.226 -0.002 -0.996 -0.    -0.    -0.     0.   ]
 [ 0.945  0.008 -0.021  0.     0.    -0.    -0.   ]
 [-0.009  1.     0.    -0.    -0.    -0.    -0.   ]
 [ 0.    -0.    -0.     0.     0.     0.     0.   ]
 [ 0.53   0.005 -0.01   0.     0.    -0.    -0.   ]
 [-0.148 -0.001  0.015 -0.    -0.     0.    -0.   ]
 [-0.148 -0.001  0.015 -0.    -0.     0.    -0.   ]]


# checking orb E

In [72]:
s_mat = driver._global_rks.get_ovlp()
s_half = sp.linalg.fractional_matrix_power(s_mat, 0.5)
C_all_localized_and_virt = driver.localized_system.c_loc_occ_and_virt

# find orthogonal orbitals
ortho_std = s_half @ driver._global_rks.mo_coeff
ortho_loc = s_half @ C_all_localized_and_virt

# Build change of basis operator (maps between orthonormal basis (canonical and localized)
unitary_ORTHO_std_onto_loc = np.einsum("ik,jk->ij", ortho_std, ortho_loc)

print('is unitary:', np.allclose(unitary_ORTHO_std_onto_loc@unitary_ORTHO_std_onto_loc.conj().T,
           np.eye(unitary_ORTHO_std_onto_loc.shape[0])))

is unitary: True


In [73]:
print('U |loc_ortho> = |std_ortho>', np.allclose(unitary_ORTHO_std_onto_loc @ ortho_loc, 
                    ortho_std))

U |loc_ortho> = |std_ortho> True


In [74]:
s_neg_half = sp.linalg.fractional_matrix_power(s_mat, -0.5)
matrix_std_to_loc = s_neg_half @ unitary_ORTHO_std_onto_loc @ s_half

print('V |loc> = |std>', np.allclose(matrix_std_to_loc @ C_all_localized_and_virt, 
                    driver._global_rks.mo_coeff))



V |loc> = |std> True


In [75]:
dm_rks = driver._global_rks.make_rdm1(
             mo_coeff=C_all_localized_and_virt, 
             mo_occ=driver._global_rks.mo_occ)

fock_mat = driver._global_rks.get_fock(dm=dm_rks)

fock_mat_loc_basis = matrix_std_to_loc.conj().T @ fock_mat @ matrix_std_to_loc

np.around(C_all_localized_and_virt.conj().T @ fock_mat_loc_basis @ C_all_localized_and_virt, 3)


array([[-18.835,   0.   ,  -0.   ,   0.   ,   0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.931,   0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ],
       [ -0.   ,  -0.   ,  -0.433,   0.   ,  -0.   ,  -0.   ,  -0.   ],
       [  0.   ,  -0.   ,   0.   ,  -0.234,   0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.142,   0.   ,  -0.   ],
       [  0.   ,  -0.   ,  -0.   ,   0.   ,   0.   ,   0.356,  -0.   ],
       [ -0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ,   0.462]])

In [76]:
P_mu = driver.mu_level_shift * driver._env_projector

# Fock + mu_shift
np.around(C_all_localized_and_virt.conj().T @ (fock_mat_loc_basis+P_mu) @ C_all_localized_and_virt, 3)

array([[-1.88350000e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -9.31000000e-01, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         0.00000000e+00],
       [-0.00000000e+00, -0.00000000e+00, -4.33000000e-01,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         1.99999977e+06, -0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  1.99999986e+06,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         0.00000000e+00, -0.00000000e+00,  3.56000000e-01,
        -0.00000000e+00],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         4.6200000

In [77]:
dm_env = driver.localized_system.dm_enviro
dm_env= driver.localized_system.c_enviro @ driver.localized_system.c_enviro.T # <-- this is likely why! 

In [102]:
dm_env = driver.localized_system.dm_enviro

fock_mat = driver._global_rks.get_fock(dm=dm_rks)
fock_mat_loc_basis = matrix_std_to_loc.conj().T @ fock_mat @ matrix_std_to_loc


FDS = fock_mat_loc_basis @ dm_env @ s_mat

# with 0.25 rather than 0.5 (env MO energies are zeroed out)
huzinaga_op_std = -0.5 * (FDS + FDS.T) 
# with 0.5 they are pushed to MO energy*2 (aka ensuring always +ve so never filled)


# huzinaga_op_std = np.zeros_like(huzinaga_op_std)

np.around(C_all_localized_and_virt.conj().T @ (fock_mat_loc_basis+huzinaga_op_std) @ C_all_localized_and_virt, 3)
# np.around(C_all_localized_and_virt.conj().T @ (fock_mat_loc_basis) @ C_all_localized_and_virt, 3)

array([[-18.835,   0.   ,  -0.   ,  -0.   ,  -0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.931,   0.   ,   0.   ,   0.   ,  -0.   ,  -0.   ],
       [ -0.   ,  -0.   ,  -0.433,   0.   ,  -0.   ,  -0.   ,  -0.   ],
       [  0.   ,  -0.   ,   0.   ,   0.234,   0.   ,   0.   ,   0.   ],
       [ -0.   ,  -0.   ,  -0.   ,   0.   ,   0.142,  -0.   ,  -0.   ],
       [  0.   ,  -0.   ,   0.   ,   0.   ,   0.   ,   0.356,   0.   ],
       [ -0.   ,  -0.   ,  -0.   ,   0.   ,  -0.   ,  -0.   ,   0.462]])

In [103]:
np.diag(C_all_localized_and_virt.conj().T @ (fock_mat_loc_basis+huzinaga_op_std) @ C_all_localized_and_virt)

array([-18.83525318,  -0.93132671,  -0.43263415,   0.23435814,
         0.14179089,   0.35582625,   0.46190031])

In [98]:
np.around(C_all_localized_and_virt.conj().T @ (huzinaga_op_std) @ C_all_localized_and_virt, 3)

array([[-0.   , -0.   , -0.   , -0.   , -0.   ,  0.   , -0.   ],
       [-0.   , -0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   ],
       [-0.   , -0.   , -0.   , -0.   ,  0.   ,  0.   ,  0.   ],
       [-0.   ,  0.   , -0.   ,  0.469,  0.   , -0.   ,  0.   ],
       [-0.   ,  0.   ,  0.   ,  0.   ,  0.284, -0.   ,  0.   ],
       [ 0.   , -0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ],
       [-0.   , -0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   ]])

In [99]:
# see how in above diagonal matrix how when huzinaga_op_std is included the 
# env orbitals are positive what they were before...
np.around(C_all_localized_and_virt.conj().T @ fock_mat_loc_basis @ C_all_localized_and_virt, 3)

array([[-18.835,   0.   ,  -0.   ,   0.   ,   0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.931,   0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ],
       [ -0.   ,  -0.   ,  -0.433,   0.   ,  -0.   ,  -0.   ,  -0.   ],
       [  0.   ,  -0.   ,   0.   ,  -0.234,   0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.142,   0.   ,  -0.   ],
       [  0.   ,  -0.   ,  -0.   ,   0.   ,   0.   ,   0.356,  -0.   ],
       [ -0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ,   0.462]])

In [100]:
# huzinaga_op_std = -1* (FDS + FDS.T)

# scf_method_HUZ.get_fock() @ huzinaga_op_std - huzinaga_op_std @ scf_method_HUZ.get_fock()

In [101]:
proj_exact = localized_system.dm_enviro/2 @ s_mat

new_op = fock_mat_loc_basis - proj_exact@fock_mat_loc_basis - fock_mat_loc_basis@proj_exact


np.around(C_all_localized_and_virt.conj().T @ fock_mat_loc_basis @ C_all_localized_and_virt, 3)

array([[-18.835,   0.   ,  -0.   ,   0.   ,   0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.931,   0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ],
       [ -0.   ,  -0.   ,  -0.433,   0.   ,  -0.   ,  -0.   ,  -0.   ],
       [  0.   ,  -0.   ,   0.   ,  -0.234,   0.   ,   0.   ,  -0.   ],
       [  0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.142,   0.   ,  -0.   ],
       [  0.   ,  -0.   ,  -0.   ,   0.   ,   0.   ,   0.356,  -0.   ],
       [ -0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ,  -0.   ,   0.462]])

In [84]:
proj_exact = localized_system.dm_enviro/2 @ s_mat

huz =  fock_mat_loc_basis - proj_exact@fock_mat_loc_basis - fock_mat_loc_basis@proj_exact


np.around(C_all_localized_and_virt.conj().T @ huz @ C_all_localized_and_virt, 3)

array([[-1.893e+01, -0.000e+00, -2.000e-03, -1.100e-02,  0.000e+00,
        -2.000e-03, -0.000e+00],
       [-0.000e+00, -9.310e-01, -0.000e+00, -0.000e+00, -0.000e+00,
        -0.000e+00, -0.000e+00],
       [-3.800e-01, -0.000e+00, -4.420e-01, -4.400e-02,  0.000e+00,
        -9.000e-03, -0.000e+00],
       [ 2.196e+00,  0.000e+00,  5.500e-02,  2.550e-01,  0.000e+00,
         5.100e-02,  0.000e+00],
       [ 0.000e+00, -0.000e+00, -0.000e+00,  0.000e+00,  1.430e-01,
         0.000e+00,  2.900e-02],
       [ 9.720e-01,  0.000e+00,  2.400e-02,  1.130e-01,  0.000e+00,
         3.780e-01,  0.000e+00],
       [ 0.000e+00, -0.000e+00, -0.000e+00,  0.000e+00,  2.700e-02,
        -0.000e+00,  4.670e-01]])

In [85]:
driver._env_projector @ huz - huz@driver._env_projector

array([[ 4.80325420e-03,  2.06081617e-01, -1.77490916e-17,
         6.46550923e-07, -2.38807639e-01,  2.27390055e-01,
         2.27389297e-01],
       [ 6.93835540e-02,  2.97687244e+00, -2.46141210e-16,
         1.07929101e-07, -3.44960355e+00,  3.28466983e+00,
         3.28466971e+00],
       [-2.87722355e-04, -1.23446076e-02,  1.05531880e-18,
        -5.26907230e-15,  1.43049469e-02, -1.36209933e-02,
        -1.36209933e-02],
       [ 1.07192306e-08,  4.59904097e-07,  4.64631551e-15,
        -1.83226706e-03, -5.32937509e-07, -1.07298927e-03,
         1.07400418e-03],
       [ 7.32047069e-02,  3.14081741e+00, -2.71451344e-16,
         8.31090289e-08, -3.63958319e+00,  3.46556607e+00,
         3.46556598e+00],
       [ 6.94862852e-03,  2.98128008e-01,  2.70076290e-15,
         1.56366411e-03, -3.45471113e-01,  3.29869440e-01,
         3.28037188e-01],
       [ 6.94864681e-03,  2.98128793e-01, -2.74020751e-15,
        -1.56368830e-03, -3.45472022e-01,  3.28038040e-01,
         3.2987032

In [86]:
proj_exact = s_mat@ localized_system.dm_enviro @ s_mat
proj_exact =  localized_system.dm_enviro/2 @ s_mat


Fock_emb =  fock_mat_loc_basis - proj_exact@fock_mat_loc_basis - fock_mat_loc_basis@proj_exact

fock_mat = driver._global_rks.get_fock(dm=dm_rks)
Fock_emb =  fock_mat - proj_exact@fock_mat - fock_mat@proj_exact

np.around(proj_exact @ Fock_emb - Fock_emb @ proj_exact, 9) #  <-commutes!

array([[ 0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0., -0.,  0., -0., -0.],
       [-0., -0.,  0.,  0., -0., -0., -0.],
       [ 0.,  0., -0., -0.,  0.,  0.,  0.],
       [-0., -0.,  0., -0., -0., -0., -0.],
       [-0.,  0., -0.,  0., -0.,  0.,  0.]])

In [87]:
proj_act = localized_system.dm_active/2 @ s_mat # uses active dmat!

print(np.around(localized_system.c_active, 3))
print()
print(np.around(proj_act @ localized_system.c_loc_occ_and_virt, 3))

[[-0.226 -0.002 -0.996]
 [ 0.945  0.008 -0.021]
 [-0.009  1.    -0.   ]
 [ 0.    -0.    -0.   ]
 [ 0.53   0.005 -0.01 ]
 [-0.148 -0.001  0.015]
 [-0.148 -0.001  0.015]]

[[-0.226 -0.002 -0.996 -0.    -0.    -0.     0.   ]
 [ 0.945  0.008 -0.021  0.     0.    -0.    -0.   ]
 [-0.009  1.     0.    -0.    -0.    -0.    -0.   ]
 [ 0.    -0.    -0.     0.     0.     0.     0.   ]
 [ 0.53   0.005 -0.01   0.     0.    -0.    -0.   ]
 [-0.148 -0.001  0.015 -0.    -0.     0.    -0.   ]
 [-0.148 -0.001  0.015 -0.    -0.     0.    -0.   ]]
